In [ ]:
# import numpy as np
# import xarray as xr
# import geopandas as gpd
# import pandas as pd
# from sklearn.ensemble import RandomForestClassifier
# from skimage.transform import resize
# from pystac_client import Client
# from dask.distributed import Client as DaskClient
# from odc.stac import load, configure_s3_access
# import joblib
# import folium
# import matplotlib.pyplot as plt

In [ ]:
from pystac_client import Client
from dask.distributed import Client as DaskClient
from odc.stac import load, configure_s3_access
import geopandas as gpd
import pandas as pd
import numpy as np
import xarray as xr
import folium
import joblib
import matplotlib.pyplot as plt


from sklearn.ensemble import RandomForestClassifier

import odc.geo.xr  # noqa: F401

In [ ]:
# optional: rioxarray for saving GeoTIFF
try:
    import rioxarray  # noqa: F401
    RIO_AVAILABLE = True
except Exception:
    RIO_AVAILABLE = False

In [ ]:
# User-configurable values - edit these
STAC_CATALOG = "https://earth-search.aws.element84.com/v1"   # STAC endpoint
AOI_PATH = "AOI/EfateAOI.geojson"                           # path to your AOI GeoJSON
TRAINING_PATH = "Training_Data/InvasiveClean6.geojson"      # training points GeoJSON
DATETIME = "2024-05/2024-09"                                # date range for search
SENTINEL_COLLECTION = ["sentinel-2-c1-l2a"]

In [ ]:
# Mask/band options
MASK_NAME = "combined_mask"
INCLUDE_MASK_AS_FEATURE = False   # True to include mask as a predictor band, False to only use to exclude pixels


In [ ]:
# Model / outputs
RF_N_ESTIMATORS = 100
RF_RANDOM_STATE = 42
MODEL_OUT = "rf_model.joblib"
PRED_GTIFF = "predicted.tif"
OUT_DS_NETCDF = "predicted_dataset.nc"

In [ ]:
# Small helper for debugging prints
def info(msg, *args):
    print("[INFO]", msg, *args)

In [ ]:
info("Opening STAC catalog:", STAC_CATALOG)
catalog = Client.open(STAC_CATALOG)

In [ ]:
# Define bounding box – should match your area
aoi_gdf = gpd.read_file("AOI/EfateAOI.geojson")
aoi_gdf.plot(edgecolor="red", facecolor="none")
plt.title("AOI Check")
plt.show()  # Replace with actual coordinates

In [ ]:
# Cell A: Diagnose why geopandas can't find the file
import os
from pathlib import Path

AOI_PATH = "Fiji FIT/AOI/EfateAOI.geojson"   # your current value

print("Current working directory:", os.getcwd())
print("Absolute path for AOI_PATH:", Path(AOI_PATH).resolve())
print("Exists?", Path(AOI_PATH).exists())

# List files in current dir and the AOI folder (if present)
print("\nTop-level files/directories:")
print(sorted(os.listdir("."))[:200])

aoi_dir = Path(AOI_PATH).parent
if aoi_dir.exists():
    print(f"\nFiles in {aoi_dir}:")
    print(sorted(os.listdir(aoi_dir))[:200])
else:
    print(f"\nDirectory {aoi_dir} does not exist")

In [ ]:
# Define bounding box – should match your area
aoi_gdf = gpd.read_file("AOI/EfateAOI.geojson")
aoi_gdf.plot(edgecolor="red", facecolor="none")
plt.title("AOI Check")
plt.show()  # Replace with actual coordinates

In [ ]:
# Load AOI
aoi_gdf = gpd.read_file(AOI_PATH)
info("AOI bounds:", aoi_gdf.total_bounds)
aoi_gdf.plot(edgecolor="red", facecolor="none")
plt.title("AOI Check")
plt.show()